In [26]:
import pandas as pd

### db 연동 (mysql)

In [ ]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL 연결 정보
HOST = "localhost"  # 또는 "127.0.0.1"
USER = "root"
PASSWORD = "toor"
DATABASE = "zipcoding"

# SQLAlchemy 엔진 생성 (Pandas to_sql을 사용하기 위함)
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DATABASE}")

# mysql.connector를 사용한 연결 (커서 사용 가능)
conn = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
cursor = conn.cursor()


### db에 2019~2025 데이터 올리기

In [16]:
df2019 = pd.read_csv("C:/ZIPcoding/original_data/서울시 부동산 실거래가 정보 2019.csv", encoding = 'euc-kr', low_memory=False)
df2020 = pd.read_csv("C:/ZIPcoding/original_data/서울시 부동산 실거래가 정보 2020.csv", encoding = 'euc-kr', low_memory=False)
df2021 = pd.read_csv("C:/ZIPcoding/original_data/서울시 부동산 실거래가 정보 2021.csv", encoding = 'euc-kr', low_memory=False)
df2022 = pd.read_csv("C:/ZIPcoding/original_data/서울시 부동산 실거래가 정보 2022.csv", encoding = 'euc-kr', low_memory=False)
df2023 = pd.read_csv("C:/ZIPcoding/original_data/서울시 부동산 실거래가 정보 2023.csv", encoding = 'euc-kr', low_memory=False)
df2024 = pd.read_csv("C:/ZIPcoding/original_data/서울시 부동산 실거래가 정보 2024.csv", encoding = 'euc-kr', low_memory=False)
df2025 = pd.read_csv("C:/ZIPcoding/original_data/서울시 부동산 실거래가 정보 2025.csv", encoding = 'euc-kr', low_memory=False)

In [ ]:
# 여러 개의 DataFrame을 리스트로 묶어 한 번에 병합
df_list = [df2025, df2024, df2023, df2022, df2021, df2020, df2019]
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df.to_sql(name="original_data", con=engine, if_exists="append", index=True)

In [10]:
df.columns

Index(['접수연도', '자치구코드', '자치구명', '법정동코드', '법정동명', '지번구분', '지번구분명', '본번', '부번',
       '건물명', '계약일', '물건금액(만원)', '건물면적(㎡)', '토지면적(㎡)', '층', '권리구분', '취소일',
       '건축년도', '건물용도', '신고구분', '신고한 개업공인중개사 시군구명', '건물면적정수', '이전 실거래가(만원)'],
      dtype='object')

In [12]:
## 컬럼명 때문에 오류 나서 괄호들, 띄어쓰기들 다 뺌

df.columns =  ['접수연도', '자치구코드', '자치구명', '법정동코드', '법정동명', '지번구분', '지번구분명', '본번', '부번',
       '건물명', '계약일', '물건금액', '건물면적', '토지면적', '층', '권리구분', '취소일',
       '건축년도', '건물용도', '신고구분', '신고한개업공인중개사시군구명']

### 자치구, 법정동별 6개월 거래량, 평균 거래금액 계산해서 파일 저장

In [14]:
import pandas as pd
import mysql.connector

for i in range(2020, 2026):
    # ✅ MySQL 연결 설정
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="toor",
        database="zipcoding"
    )
    
    query = f"""
       WITH 거래량_평균 AS (
        SELECT
            인덱스,
            계약일,
            물건금액,
            자치구코드,
            법정동코드,
    
            -- ✅ 자치구 기준 최근 6개월 거래량
            COUNT(물건금액) OVER (
                PARTITION BY 자치구코드
                ORDER BY 계약일 
                RANGE BETWEEN INTERVAL 6 MONTH PRECEDING AND CURRENT ROW
            ) AS 자치구최근6개월거래량,
    
            -- ✅ 자치구 기준 최근 6개월 평균 물건금액
            AVG(물건금액) OVER (
                PARTITION BY 자치구코드
                ORDER BY 계약일 
                RANGE BETWEEN INTERVAL 6 MONTH PRECEDING AND CURRENT ROW
            ) AS 자치구최근6개월평균금액,
    
            -- ✅ 자치구 + 법정동 기준 최근 6개월 거래량
            COUNT(물건금액) OVER (
                PARTITION BY 자치구코드, 법정동코드
                ORDER BY 계약일 
                RANGE BETWEEN INTERVAL 6 MONTH PRECEDING AND CURRENT ROW
            ) AS 동최근6개월거래량,
    
            -- ✅ 자치구 + 법정동 기준 최근 6개월 평균 물건금액
            AVG(물건금액) OVER (
                PARTITION BY 자치구코드, 법정동코드
                ORDER BY 계약일 
                RANGE BETWEEN INTERVAL 6 MONTH PRECEDING AND CURRENT ROW
            ) AS 동최근6개월평균금액
    
        FROM original_data
    )
    SELECT * FROM 거래량_평균
    WHERE YEAR(계약일) = {i}  
    AND 동최근6개월거래량 IS NOT NULL  -- ✅ 법정동 데이터에서 거래가 있는 행만 유지
    ORDER BY 계약일 DESC;
    
        
    """
    
    try:
        # ✅ MySQL 데이터 조회 후 DataFrame으로 변환
        df = pd.read_sql(query, conn)
    
        # ✅ CSV로 저장
        df.to_csv(f"C:/ZIPcoding/region_output_{i}.csv", index=False, encoding="utf-8-sig")
    
        # ✅ Excel 파일로 저장 (선택)
        # df.to_excel("C:/ZIPcoding/output_2020.xlsx", index=False, encoding="utf-8-sig")
    
        print("✅ 데이터 저장 완료!")
    
    except Exception as e:
        print(f"🚨 오류 발생: {e}")
    
    finally:
        # ✅ MySQL 연결 종료
        conn.close()
        print("✅ MySQL 연결 종료됨")


C:\Users\user\AppData\Local\Temp\ipykernel_13272\1672793738.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ 데이터 저장 완료!
✅ MySQL 연결 종료됨
✅ 데이터 저장 완료!
✅ MySQL 연결 종료됨
✅ 데이터 저장 완료!
✅ MySQL 연결 종료됨
✅ 데이터 저장 완료!
✅ MySQL 연결 종료됨
✅ 데이터 저장 완료!
✅ MySQL 연결 종료됨
✅ 데이터 저장 완료!
✅ MySQL 연결 종료됨


### original_data 데이터셋 불러오기

In [83]:
original_data = pd.read_csv("C:/ZIPcoding/2019~2025 data.csv", low_memory=False)

In [8]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689611 entries, 0 to 689610
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   접수연도              689611 non-null  int64  
 1   자치구코드             689611 non-null  int64  
 2   자치구명              689610 non-null  object 
 3   법정동코드             689611 non-null  int64  
 4   법정동명              689611 non-null  object 
 5   지번구분              646137 non-null  float64
 6   지번구분명             646137 non-null  object 
 7   본번                646149 non-null  object 
 8   부번                646149 non-null  object 
 9   건물명               646142 non-null  object 
 10  계약일               689611 non-null  int64  
 11  물건금액(만원)          689611 non-null  int64  
 12  건물면적(㎡)           689611 non-null  float64
 13  토지면적(㎡)           550039 non-null  float64
 14  층                 646196 non-null  float64
 15  권리구분              5189 non-null    object 
 16  취소일               27

### 6개월 치 계산한 데이터 불러오기

In [84]:
region_output2020 = pd.read_csv("C:/ZIPcoding/region_output_2020.csv", low_memory=False)
region_output2021 = pd.read_csv("C:/ZIPcoding/region_output_2021.csv", low_memory=False)
region_output2022 = pd.read_csv("C:/ZIPcoding/region_output_2022.csv", low_memory=False)
region_output2023 = pd.read_csv("C:/ZIPcoding/region_output_2023.csv", low_memory=False)
region_output2024 = pd.read_csv("C:/ZIPcoding/region_output_2024.csv", low_memory=False)
region_output2025 = pd.read_csv("C:/ZIPcoding/region_output_2025.csv", low_memory=False)

In [75]:
region_output2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64820 entries, 0 to 64819
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   인덱스           64820 non-null  int64  
 1   계약일           64820 non-null  object 
 2   물건금액          64820 non-null  int64  
 3   자치구코드         64820 non-null  int64  
 4   법정동코드         64820 non-null  int64  
 5   자치구최근6개월거래량   64820 non-null  int64  
 6   자치구최근6개월평균금액  64820 non-null  float64
 7   동최근6개월거래량     64820 non-null  int64  
 8   동최근6개월평균금액    64820 non-null  float64
dtypes: float64(2), int64(6), object(1)
memory usage: 4.5+ MB


### 데이터 병합

In [97]:
all = pd.concat([region_output2020, region_output2021, region_output2022, region_output2023, region_output2024, region_output2025])

In [103]:
all.iloc[:,[0,5,6,7,8]]

인덱스  자치구최근6개월거래량  자치구최근6개월평균금액  동최근6개월거래량   동최근6개월평균금액
0     373367         4305    66288.2381       1223   47385.3998
1     373445         4305    66288.2381       1223   47385.3998
2     373530         4305    66288.2381       1223   47385.3998
3     373553         4305    66288.2381        118   75381.2712
4     373557         4305    66288.2381        118   75381.2712
...      ...          ...           ...        ...          ...
6115    6118         2336    99489.8288         90   82274.2222
6116    6098         2336    99489.8288         51   89870.5882
6117    6112         2114    95035.8832        157  105898.1529
6118    6102         2206   194465.3395        107  128128.0374
6119    6107         2708   213764.9830        179  187314.2291

[548407 rows x 5 columns]

In [117]:
result = pd.merge(original_data, all.iloc[:,[0,5,6,7,8]], left_index = True, right_on = '인덱스', how='left')

In [119]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 689611 entries, 10.0 to nan
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   접수연도              689611 non-null  int64  
 1   자치구코드             689611 non-null  int64  
 2   자치구명              689610 non-null  object 
 3   법정동코드             689611 non-null  int64  
 4   법정동명              689611 non-null  object 
 5   지번구분              646137 non-null  float64
 6   지번구분명             646137 non-null  object 
 7   본번                646149 non-null  object 
 8   부번                646149 non-null  object 
 9   건물명               646142 non-null  object 
 10  계약일               689611 non-null  int64  
 11  물건금액(만원)          689611 non-null  int64  
 12  건물면적(㎡)           689611 non-null  float64
 13  토지면적(㎡)           550039 non-null  float64
 14  층                 646196 non-null  float64
 15  권리구분              5189 non-null    object 
 16  취소일               27357 n

In [121]:
result.sort_index()

접수연도  자치구코드  자치구명  법정동코드 법정동명  지번구분 지번구분명      본번    부번            건물명  \
0.0  2025  11440   마포구  10400  도화동   1.0    대지   550.0   0.0             삼성   
0.0  2025  11740   강동구  10900  천호동   1.0    대지   227.0   2.0          아성플러스   
0.0  2023  11740   강동구  10900  천호동   1.0    대지   110.0   3.0         세림힐하우스   
0.0  2024  11680   강남구  10300  개포동   1.0    대지    0012  0000        삼익대청아파트   
0.0  2022  11440   마포구  12500  성산동   1.0    대지   271.0  13.0           유성빌라   
..    ...    ...   ...    ...  ...   ...   ...     ...   ...            ...   
NaN  2018  11500   강서구  10300  화곡동   1.0    대지  1159.0   0.0  우장산아이파크,이편한세상   
NaN  2018  11410  서대문구  11200  대현동   1.0    대지   144.0   0.0           럭키대현   
NaN  2018  11590   동작구  10700  사당동   1.0    대지   169.0  32.0             현대   
NaN  2018  11350   노원구  10500  상계동   1.0    대지  1380.0   4.0           초산빌라   
NaN  2018  11380   은평구  11000  증산동   1.0    대지   189.0  63.0   궁전맨션(189-63)   

     ...   건물용도  신고구분  신고한 개업공인중개사 시군구명  건물면적정수  이전 실거래가(만원)     인덱스  \
0.0  ...    아파트   직거래               NaN      72     129500.0       1   
0.0  ...  연립다세대   직거래               NaN      24      37000.0    6234   
0.0  ...  연립다세대  중개거래            서울 강동구      34          NaN  173530   
0.0  ...    아파트  중개거래            서울 강남구      60          NaN  103044   
0.0  ...  연립다세대  중개거래            서울 마포구      53       5300.0  238405   
..   ...    ...   ...               ...     ...          ...     ...   
NaN  ...    아파트   NaN               NaN     101      90000.0  689606   
NaN  ...    아파트   NaN               NaN      83      76000.0  689607   
NaN  ...    아파트   NaN               NaN      51      41800.0  689608   
NaN  ...  연립다세대   NaN               NaN      41          NaN  689609   
NaN  ...  연립다세대   NaN               NaN      54      19500.0  689610   

     자치구최근6개월거래량  자치구최근6개월평균금액 동최근6개월거래량   동최근6개월평균금액  
0.0       1881.0    86733.8729     139.0   85297.1223  
0.0       2751.0    83082.8241     509.0   51100.5010  
0.0       1003.0    48515.3370     330.0   36431.8939  
0.0       1784.0   185502.4254     278.0  181095.0504  
0.0       2543.0    63767.6217     802.0   45250.2781  
..           ...           ...       ...          ...  
NaN          NaN           NaN       NaN          NaN  
NaN          NaN           NaN       NaN          NaN  
NaN          NaN           NaN       NaN          NaN  
NaN          NaN           NaN       NaN          NaN  
NaN          NaN           NaN       NaN          NaN  

[689611 rows x 28 columns]

### 원래의 인덱스 값을 담고 있는 '인덱스' 컬럼을 index 설정하고 정렬

In [125]:
result = result.set_index('인덱스').sort_index()

In [127]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 689611 entries, 0 to 689610
Data columns (total 27 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   접수연도              689611 non-null  int64  
 1   자치구코드             689611 non-null  int64  
 2   자치구명              689610 non-null  object 
 3   법정동코드             689611 non-null  int64  
 4   법정동명              689611 non-null  object 
 5   지번구분              646137 non-null  float64
 6   지번구분명             646137 non-null  object 
 7   본번                646149 non-null  object 
 8   부번                646149 non-null  object 
 9   건물명               646142 non-null  object 
 10  계약일               689611 non-null  int64  
 11  물건금액(만원)          689611 non-null  int64  
 12  건물면적(㎡)           689611 non-null  float64
 13  토지면적(㎡)           550039 non-null  float64
 14  층                 646196 non-null  float64
 15  권리구분              5189 non-null    object 
 16  취소일               27357 n

In [141]:
result[result['법정동명']=='신사동'][['계약일', '자치구최근6개월거래량']]

계약일  자치구최근6개월거래량
인덱스                          
206     20250210       2115.0
364     20250207       2144.0
688     20250205       2167.0
770     20250204       2167.0
856     20250203       2194.0
...          ...          ...
689290  20190103          NaN
689335  20190103          NaN
689349  20190103          NaN
689455  20190102          NaN
689510  20190102          NaN

[4875 rows x 2 columns]

In [143]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 689611 entries, 0 to 689610
Data columns (total 27 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   접수연도              689611 non-null  int64  
 1   자치구코드             689611 non-null  int64  
 2   자치구명              689610 non-null  object 
 3   법정동코드             689611 non-null  int64  
 4   법정동명              689611 non-null  object 
 5   지번구분              646137 non-null  float64
 6   지번구분명             646137 non-null  object 
 7   본번                646149 non-null  object 
 8   부번                646149 non-null  object 
 9   건물명               646142 non-null  object 
 10  계약일               689611 non-null  int64  
 11  물건금액(만원)          689611 non-null  int64  
 12  건물면적(㎡)           689611 non-null  float64
 13  토지면적(㎡)           550039 non-null  float64
 14  층                 646196 non-null  float64
 15  권리구분              5189 non-null    object 
 16  취소일               27357 n

### 데이터 저장

In [145]:
result.to_csv("C:/ZIPcoding/dataset0217.csv")

### 2019년 데이터 삭제하고 저장

In [147]:
# ✅ 계약일 컬럼에서 2020년 이후 데이터만 남기기
result = result[result["계약일"].astype(str).str[:4].astype(int) >= 2020]

# ✅ 최종 결과 확인
print(f"✅ 2020년 이후 데이터 필터링 완료! 총 {result.shape[0]} 행")
print(result.head())

✅ 2020년 이후 데이터 필터링 완료! 총 548407 행
     접수연도  자치구코드  자치구명  법정동코드 법정동명  지번구분 지번구분명     본번    부번    건물명  ...  \
인덱스                                                                 ...   
0    2025  11230  동대문구  10400  전농동   1.0    대지    3.0  92.0  이도인빌딩  ...   
1    2025  11440   마포구  10400  도화동   1.0    대지  550.0   0.0     삼성  ...   
2    2025  11230  동대문구  10400  전농동   1.0    대지    3.0  92.0  이도인빌딩  ...   
3    2025  11230  동대문구  10400  전농동   1.0    대지    3.0  92.0  이도인빌딩  ...   
4    2025  11230  동대문구  10400  전농동   1.0    대지    3.0  92.0  이도인빌딩  ...   

       건축년도   건물용도  신고구분  신고한 개업공인중개사 시군구명  건물면적정수 이전 실거래가(만원)  자치구최근6개월거래량  \
인덱스                                                                           
0    2018.0   오피스텔   직거래               NaN      26      6991.0       1827.0   
1    1997.0    아파트   직거래               NaN      72    129500.0       1881.0   
2    2018.0  연립다세대   직거래               NaN      34     10463.0       1827.0   
3    2018.0  연립다세대   직거래               NaN   

In [149]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 548407 entries, 0 to 563537
Data columns (total 27 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   접수연도              548407 non-null  int64  
 1   자치구코드             548407 non-null  int64  
 2   자치구명              548406 non-null  object 
 3   법정동코드             548407 non-null  int64  
 4   법정동명              548407 non-null  object 
 5   지번구분              515135 non-null  float64
 6   지번구분명             515135 non-null  object 
 7   본번                515147 non-null  object 
 8   부번                515147 non-null  object 
 9   건물명               515132 non-null  object 
 10  계약일               548407 non-null  int64  
 11  물건금액(만원)          548407 non-null  int64  
 12  건물면적(㎡)           548407 non-null  float64
 13  토지면적(㎡)           476004 non-null  float64
 14  층                 515186 non-null  float64
 15  권리구분              3076 non-null    object 
 16  취소일               27357 n

In [151]:
result.to_csv("C:/ZIPcoding/dataset0217ver.csv")